In [54]:
import pandas as pd
from sklearn.preprocessing import label_binarize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier

import warnings
warnings.filterwarnings("ignore")
#Please ignore the warnings with version change

from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive



Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [55]:
trainfile = r'/gdrive/My Drive/CIS 508/Insurance Fraud - TRAIN-3000(1).csv'
trainData = pd.read_csv(trainfile) #creates a dataframe
testfile = r'/gdrive/My Drive/CIS 508/Insurance Fraud -TEST-12900(1).csv'
testData = pd.read_csv(testfile) #creates a dataframe

#print sizes (shape) of datasets
print(trainData.shape)
print(testData.shape)

trainData.head()
testData.head()

(2999, 32)
(12918, 32)


,MONTH,WEEKOFMONTH,DAYOFWEEK,MAKE,ACCIDENTAREA,DAYOFWEEKCLAIMED,MONTHCLAIMED,WEEKOFMONTHCLAIMED,SEX,MARITALSTATUS,AGE,FAULT,POLICYTYPE,VEHICLECATEGORY,VEHICLEPRICE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,DAYS_POLICY_ACCIDENT,DAYS_POLICY_CLAIM,PASTNUMBEROFCLAIMS,AGEOFVEHICLE,AGEOFPOLICYHOLDER,POLICEREPORTFILED,WITNESSPRESENT,AGENTTYPE,NUMBEROFSUPPLIMENTS,ADDRESSCHANGE_CLAIM,NUMBEROFCARS,YEAR,BASEPOLICY,FRAUDFOUND
0,Jul,3,Sunday,Honda,Rural,Wednesday,Jan,4,Male,Married,21,Policy_Holder,Sport-Collision,Sport,more_than_69000,4,400,4,more_than_30,more_than_30,none,4_years,26_to_30,No,No,External,3_to_5,no_change,1-vehicle,1994,Collision,Yes
1,Nov,5,Monday,Mazda,Urban,Wednesday,Dec,1,Male,Single,68,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,9,400,3,more_than_30,more_than_30,2_to_4,5_years,over_65,No,No,External,none,no_change,1-vehicle,1994,All_Perils,Yes
2,Jan,1,Monday,Pontiac,Urban,Wednesday,Jan,1,Male,Married,50,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,8,400,2,more_than_30,more_than_30,none,7_years,41_to_50,No,No,External,none,under_6_months,1-vehicle,1994,All_Perils,Yes
3,Dec,1,Monday,Toyota,Rural,Tuesday,May,3,Male,Married,39,Policy_Holder,Sedan-All_Perils,Sedan,30000_to_39000,1,400,3,more_than_30,more_than_30,none,more_than_7,36_to_40,No,No,External,more_than_5,under_6_months,2-vehicles,1994,All_Perils,Yes
4,Dec,5,Wednesday,Pontiac,Urban,Wednesday,Jan,1,Male,Single,43,Policy_Holder,Sedan-Collision,Sedan,40000_to_59000,1,400,4,more_than_30,more_than_30,2_to_4,7_years,36_to_40,No,No,External,more_than_5,no_change,1-vehicle,1994,Collision,Yes


In [56]:
#Copy Train data excluding target
trainData_Copy = trainData.iloc[:, :-1].copy()
testData_Copy = testData.iloc[:, :-1].copy()

trainData_Copy.head()

,MONTH,WEEKOFMONTH,DAYOFWEEK,MAKE,ACCIDENTAREA,DAYOFWEEKCLAIMED,MONTHCLAIMED,WEEKOFMONTHCLAIMED,SEX,MARITALSTATUS,AGE,FAULT,POLICYTYPE,VEHICLECATEGORY,VEHICLEPRICE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,DAYS_POLICY_ACCIDENT,DAYS_POLICY_CLAIM,PASTNUMBEROFCLAIMS,AGEOFVEHICLE,AGEOFPOLICYHOLDER,POLICEREPORTFILED,WITNESSPRESENT,AGENTTYPE,NUMBEROFSUPPLIMENTS,ADDRESSCHANGE_CLAIM,NUMBEROFCARS,YEAR,BASEPOLICY
0,Jul,3,Sunday,Honda,Rural,Wednesday,Jan,4,Male,Married,21,Policy_Holder,Sport-Collision,Sport,more_than_69000,4,400,4,more_than_30,more_than_30,none,4_years,26_to_30,No,No,External,3_to_5,no_change,1-vehicle,1994,Collision
1,Nov,5,Monday,Mazda,Urban,Wednesday,Dec,1,Male,Single,68,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,9,400,3,more_than_30,more_than_30,2_to_4,5_years,over_65,No,No,External,none,no_change,1-vehicle,1994,All_Perils
2,Jan,1,Monday,Pontiac,Urban,Wednesday,Jan,1,Male,Married,50,Policy_Holder,Sedan-All_Perils,Sedan,20000_to_29000,8,400,2,more_than_30,more_than_30,none,7_years,41_to_50,No,No,External,none,under_6_months,1-vehicle,1994,All_Perils
3,Dec,1,Monday,Toyota,Rural,Tuesday,May,3,Male,Married,39,Policy_Holder,Sedan-All_Perils,Sedan,30000_to_39000,1,400,3,more_than_30,more_than_30,none,more_than_7,36_to_40,No,No,External,more_than_5,under_6_months,2-vehicles,1994,All_Perils
4,Dec,5,Wednesday,Pontiac,Urban,Wednesday,Jan,1,Male,Single,43,Policy_Holder,Sedan-Collision,Sedan,40000_to_59000,1,400,4,more_than_30,more_than_30,2_to_4,7_years,36_to_40,No,No,External,more_than_5,no_change,1-vehicle,1994,Collision


In [57]:
#List of Categorical Features
categoricalFeatures = ["MONTH", "DAYOFWEEK", 'MAKE', "ACCIDENTAREA", "DAYOFWEEKCLAIMED", "MONTHCLAIMED", 'SEX', 'MARITALSTATUS','FAULT','POLICYTYPE','VEHICLECATEGORY','VEHICLEPRICE','DAYS_POLICY_ACCIDENT','DAYS_POLICY_CLAIM','PASTNUMBEROFCLAIMS','AGEOFVEHICLE','AGEOFPOLICYHOLDER','POLICEREPORTFILED','WITNESSPRESENT','AGENTTYPE','NUMBEROFSUPPLIMENTS','ADDRESSCHANGE_CLAIM','NUMBEROFCARS','BASEPOLICY']

#Combine Train and test for one Hot Encoding
combined_Data = pd.concat([trainData_Copy,testData_Copy], keys=[0,1])

#Do one Hot encoding for categorical features
combined_Data = pd.get_dummies(combined_Data,columns=categoricalFeatures)

#Separate Train data and test data
X_train = combined_Data.xs(0)
X_test = combined_Data.xs(1)

X_test.head()

,WEEKOFMONTH,WEEKOFMONTHCLAIMED,AGE,REPNUMBER,DEDUCTIBLE,DRIVERRATING,YEAR,MONTH_Apr,MONTH_Aug,MONTH_Dec,MONTH_Feb,MONTH_Jan,MONTH_Jul,MONTH_Jun,MONTH_Mar,MONTH_May,MONTH_Nov,MONTH_Oct,MONTH_Sep,DAYOFWEEK_Friday,DAYOFWEEK_Monday,DAYOFWEEK_Saturday,DAYOFWEEK_Sunday,DAYOFWEEK_Thursday,DAYOFWEEK_Tuesday,DAYOFWEEK_Wednesday,MAKE_Accura,MAKE_BMW,MAKE_Chevrolet,MAKE_Dodge,MAKE_Ferrari,MAKE_Ford,MAKE_Honda,MAKE_Jaguar,MAKE_Lexus,MAKE_Mazda,MAKE_Mecedes,MAKE_Mercury,MAKE_Nisson,MAKE_Pontiac,...,AGEOFVEHICLE_2_years,AGEOFVEHICLE_3_years,AGEOFVEHICLE_4_years,AGEOFVEHICLE_5_years,AGEOFVEHICLE_6_years,AGEOFVEHICLE_7_years,AGEOFVEHICLE_more_than_7,AGEOFVEHICLE_new,AGEOFPOLICYHOLDER_16_to_17,AGEOFPOLICYHOLDER_18_to_20,AGEOFPOLICYHOLDER_21_to_25,AGEOFPOLICYHOLDER_26_to_30,AGEOFPOLICYHOLDER_31_to_35,AGEOFPOLICYHOLDER_36_to_40,AGEOFPOLICYHOLDER_41_to_50,AGEOFPOLICYHOLDER_51_to_65,AGEOFPOLICYHOLDER_over_65,POLICEREPORTFILED_No,POLICEREPORTFILED_Yes,WITNESSPRESENT_No,WITNESSPRESENT_Yes,AGENTTYPE_External,AGENTTYPE_Internal,NUMBEROFSUPPLIMENTS_1_to_2,NUMBEROFSUPPLIMENTS_3_to_5,NUMBEROFSUPPLIMENTS_more_than_5,NUMBEROFSUPPLIMENTS_none,ADDRESSCHANGE_CLAIM_1_year,ADDRESSCHANGE_CLAIM_2_to_3_years,ADDRESSCHANGE_CLAIM_4_to_8_years,ADDRESSCHANGE_CLAIM_no_change,ADDRESSCHANGE_CLAIM_under_6_months,NUMBEROFCARS_1-vehicle,NUMBEROFCARS_2-vehicles,NUMBEROFCARS_3_to_4,NUMBEROFCARS_5_to_8,NUMBEROFCARS_more_than_8,BASEPOLICY_All_Perils,BASEPOLICY_Collision,BASEPOLICY_Liability
0,3,4,21,4,400,4,1994,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
1,5,1,68,9,400,3,1994,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0
2,1,1,50,8,400,2,1994,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0
3,1,3,39,1,400,3,1994,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0
4,5,1,43,1,400,4,1994,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0


In [0]:
y_train = trainData["FRAUDFOUND"]
y_test = testData["FRAUDFOUND"]

In [59]:
#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY 
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
clf_predict=clf.predict(X_test)
print("accuracy Score (training) for Decision Tree:{0:6f}".format(clf.score(X_test,y_test)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(y_test,clf_predict))



accuracy Score (training) for Decision Tree:0.885431
Confusion Matrix for Decision Tree
[[10989  1431]
 [   49   449]]


In [60]:
#Hyperparameter tuning done for decision tree classifier
#RANDOM SEARCH--------------------------------------------
print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(10,100,10),'max_depth': range(5,30,5),'criterion':['gini','entropy']}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15,cv=5)
clf_random.fit(X_train, y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#GRID SEARCH----------------------------------------
print("GridSearchCV-Decision tree")
clf_grid = GridSearchCV(clf,parameters)
clf_grid.fit(X_train, y_train)
grid_parm1=clf_grid.best_params_
print(grid_parm1)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clfr = DecisionTreeClassifier(**grid_parm1)

clf.fit(X_train,y_train)
clf_predict = clf.predict(X_test)
clfr.fit(X_train,y_train)
clfr_predict = clfr.predict(X_test)


#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning randomized search for Decision Tree:{0:6f}".format(clf.score(X_train,y_train)))

print("accuracy Score (testing) after hypertuning grid search for Decision Tree:{0:6f}".format(clfr.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(y_test,clf_predict))
print("=== Classification Report ===")
print(classification_report(y_test,clf_predict))

clf_cv_score = cross_val_score(clf, X_train, y_train, cv=10, scoring="balanced_accuracy")
print(clf_cv_score)
print('\n')




RandomizedSearchCV-Decision tree
{'min_samples_leaf': 50, 'max_depth': 15, 'criterion': 'entropy'}
GridSearchCV-Decision tree
{'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 90}
accuracy Score (training) after hypertuning randomized search for Decision Tree:0.891631
accuracy Score (testing) after hypertuning grid search for Decision Tree:0.882877
Confusion Matrix after hypertuning for Decision Tree
[[11255  1165]
 [  332   166]]
=== Classification Report ===
              precision    recall  f1-score   support

          No       0.97      0.91      0.94     12420
         Yes       0.12      0.33      0.18       498

    accuracy                           0.88     12918
   macro avg       0.55      0.62      0.56     12918
weighted avg       0.94      0.88      0.91     12918

[0.86923077 0.76153846 0.76730769 0.675      0.5875     0.5
 0.5        0.5        0.55       0.6025641 ]




In [0]:
#Normal randomforest
rand_parameters={'min_samples_leaf' : range(10,100,10),'max_depth': range(1,10,2),'max_features':[2,3,4],'n_estimators':[20,30,40]}
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
rfc_predict=rfc.predict(X_test)
print("accuracy Score (testing) for RandomForest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))

#RANDOMIZED SEARCH----------------------------------------
rfc_random = RandomizedSearchCV(rfc,rand_parameters,n_iter=25,cv=5)
rfc_random.fit(X_train, y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
print("accuracy Score (testing) after hypertuning for Random Forest:{0:6f}".format(rfc.score(X_test,y_test)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(y_test,rfc_predict))
print("=== Classification Report ===")
print(classification_report(y_test,rfc_predict))
rfc_cv_score = cross_val_score(rfc, X_train, y_train)
print(rfc_cv_score)
print('\n')




accuracy Score (testing) for RandomForest:0.953476
Confusion Matrix for Random Forest:
[[11947   473]
 [  128   370]]
